In [2]:
MINLETTERS = 3
BOARD = ["asdf","erty","iuop","abcd"]
# BOARD = ["AB","CD"]
BOARD = [word.upper() for word in BOARD]
NROWS, NCOLS = len(BOARD), len(BOARD[0])


class Node():
    def __init__(self, isWord = None):
        self.isWord = isWord
        self.children = [None]*26

    def add(self, word):
        node = self.getChild(word[0])
        if len(word) == 1:
            if node:
                node.isWord = True
            else:
                nn = Node(1)
                self.children[ord(word)-65] = nn
        else:
            if node:
                node.add(word[1:])
            else:
                nn = Node(0)
                self.children[ord(word[0])-65] = nn
                nn.add(word[1:])

    def getChild(self, chr):
        return self.children[ord(chr)-65]


class Trie():
    def __init__(self, filePath):
        self.root = Node()
        self.load(filePath)

    def load(self, filePath):
        with open(filePath) as file:
            while (word := file.readline().strip()):
                if NROWS*NCOLS >= len(word) >= MINLETTERS:
                    self.root.add(word)

trie = Trie("wordsDict.txt")


In [16]:
def score(word):
    # return 1
    if len(word) < 3:
        return 0
    if len(word) == 3:
        return 100
    if len(word) < 6:
        return 400*(len(word)-3)
    return 400*(len(word) -3)+200

def extend(root, pos, board, neighborArray):
    # print(neighborArray)
    # word, boolean visited array, trieNode
    bools = [False]*(NROWS*NCOLS)
    bools[pos] = True
    paths = [([pos], bools, root)]
    while paths:
        curr = paths.pop()
        if curr[2].isWord:
            yield ''.join(board[pos] for pos in curr[0])
        for neighborPos in neighborArray[curr[0][-1]]:
            if not curr[1][neighborPos] and (trieNode := curr[2].getChild(board[neighborPos])):
                newBools = curr[1].copy()
                newBools[neighborPos] = True
                paths.append((curr[0]+[neighborPos], newBools, trieNode))

def getNeighbors(structure):
    # Structure is a 2d list showing structure
    numberedStructure = copy.deepcopy(structure)
    num = 1
    for i, row in enumerate(structure):
        for j, v in enumerate(row):
            if v:
                numberedStructure[i][j] = num
                num += 1
    neighborsArray = []
    for i, row in enumerate(structure):
        for j, v in enumerate(row):
            if v:
                neighborsArray.append([])
                for dx, dy in [(-1,1),(-1,0),(-1,-1),(0,1),(0,-1),(1,1),(1,0),(1,-1)]:
                    if len(structure) > i+dy>= 0 and len(structure[i+dy]) > j+dx >= 0 and (v:=numberedStructure[i+dy][j+dx]):
                        neighborsArray[-1].append(v-1)
    return neighborsArray

def maxScore(board, neighbors, trie):
    words = set()
    for pos, letter in enumerate(board):
        for word in extend(trie.root.getChild(letter), pos, board, neighbors):
            words.add(word)
    points = sum(map(score, words))
    return len(words)

In [17]:
import random
import copy
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def alter(board):
    newBoard = list(board)
    for _ in range(int(random.expovariate(0.75)+1)):
        if random.random() > 0.25: # Replace letter
            p = random.randint(0,NROWS*NCOLS-1)
            newBoard[p] = random.choice(ALPHABET)
        else: # Swap two letters
            p1, p2 = random.randint(0,NROWS*NCOLS-1), random.randint(0,NROWS*NCOLS-1)
            newBoard[p1], newBoard[p2] = newBoard[p2], newBoard[p1]
    return ''.join(newBoard)

def genBoards(numBoards, structure):
    bestBoards = []
    for _ in range(numBoards):
        board = ""
        for row in structure:
            for v in row:
                if v:
                    board += random.choice(ALPHABET)
        bestBoards.append(board)
    return bestBoards

def solve(structure, numBoards = 1, iters = 10000):
    neighbors = getNeighbors(structure)
    bestBoards = genBoards(numBoards, structure)
    bestBoards = [(board, maxScore(board, neighbors, trie)) for board in bestBoards]
    for _ in range(iters):
        # if _ % 100 == 0:
        #     print("_", _)
        newBoards = []
        for board, score in bestBoards:
            newBoard = alter(board)
            if (newScore:=maxScore(newBoard, neighbors, trie)) > score:
                newBoards.append((newBoard,newScore))
            else:
                newBoards.append((board,score))
        bestBoards = newBoards
    print(*bestBoards, sep="\n")
struct = [[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]

import profile
profile.run('solve(struct, 1, 200)')

# solve(struct, 1, 5000)

# print(maxScore("SEGSRNTREIAESLPS", getNeighbors(struct), trie))
# print(maxScore("SETSPIRDLANESETS", getNeighbors(struct), trie))
# print(maxScore("SERSPATGLINESERS", getNeighbors(struct), trie))


#(['SEGSTNRTEAIEDLPS'], 1585700)



('RENDTGIRELATBDSH', 695)
         3061050 function calls (3061026 primitive calls) in 11.188 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 2018013417.py:16(genBoards)
        1    0.000    0.000   11.188   11.188 2018013417.py:27(solve)
        1    0.000    0.000    0.016    0.016 2018013417.py:30(<listcomp>)
      200    0.000    0.000    0.047    0.000 2018013417.py:5(alter)
    60893    0.359    0.000    0.578    0.000 2475932644.py:1(score)
    76534    4.703    0.000    9.906    0.000 2475932644.py:11(extend)
   393912    0.422    0.000    0.422    0.000 2475932644.py:20(<genexpr>)
        1    0.000    0.000    0.000    0.000 2475932644.py:27(getNeighbors)
      201    0.438    0.002   11.141    0.055 2475932644.py:46(maxScore)
   768666    2.469    0.000    3.219    0.000 3496535110.py:29(getChild)
     6818    0.000    0.000    0.000    0.000 :0(__setattr__)
        